In [1]:
import numpy as np
import pandas as pd
import pysal as ps
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn import metrics
from sklearn.model_selection import train_test_split

C:\Users\MSI\Anaconda3\lib\site-packages\pysal\explore\segregation\network\network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
C:\Users\MSI\Anaconda3\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

Using TensorFlow backend.


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
def dbf2DF(dbfile, upper=True): #Reads in DBF files and returns Pandas DF
    db = ps.lib.io.open(dbfile) #Pysal to open DBF
    d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
    #pandasDF = pd.DataFrame(db[:]) #Convert to Pandas DF
    pandasDF = pd.DataFrame(d) #Convert to Pandas DF
    if upper == True: #Make columns uppercase if wanted 
        pandasDF.columns = map(str.upper, db.header) 
    db.close() 
    return pandasDF

In [6]:
database = input("Enter database name: ")

Enter database name: E:\\DATASET.dbf


In [7]:
df = dbf2DF(database)

In [8]:
df = df.dropna()

In [9]:
df = df.drop(columns = ["OBJECTID","POINTID"])

In [10]:
train_df = df.loc[df["Y"] > 0]
test_df = df.loc[df["Y"] <= 0]

In [11]:
train_X_Coord = train_df["COORD_X"]
train_Y_Coord = train_df["COORD_Y"]

In [12]:
test_X_Coord = test_df["COORD_X"]
test_Y_Coord = test_df["COORD_Y"]

In [13]:
train_df = train_df.drop(columns = ["COORD_X","COORD_Y"])
test_df = test_df.drop(columns = ["COORD_X","COORD_Y"])

In [14]:
train_x = train_df.drop(columns = "Y").values
train_y = train_df["Y"].values

In [15]:
test_x = test_df.drop(columns = "Y").values

In [16]:
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.transform(test_x)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.2, random_state = 0)

In [18]:
model = Sequential()

# The Input Layer :
model.add(Dense(64, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [20]:
history = model.fit(x_train, y_train, epochs=245, batch_size=128, validation_split=0.3,  verbose=1)


Train on 59100 samples, validate on 25329 samples
Epoch 1/245
59100/59100 [==============================] - ETA: 2:12 - loss: 3.9544 - mse: 3.9544 - mae: 1.630 - ETA: 9s - loss: 3.8365 - mse: 3.8365 - mae: 1.5070  - ETA: 5s - loss: 2.6662 - mse: 2.6662 - mae: 1.211 - ETA: 3s - loss: 2.1019 - mse: 2.1019 - mae: 1.056 - ETA: 3s - loss: 1.8827 - mse: 1.8827 - mae: 0.993 - ETA: 2s - loss: 1.6439 - mse: 1.6439 - mae: 0.919 - ETA: 2s - loss: 1.4816 - mse: 1.4816 - mae: 0.865 - ETA: 1s - loss: 1.3357 - mse: 1.3357 - mae: 0.815 - ETA: 1s - loss: 1.2260 - mse: 1.2260 - mae: 0.774 - ETA: 1s - loss: 1.1404 - mse: 1.1404 - mae: 0.742 - ETA: 1s - loss: 1.0938 - mse: 1.0938 - mae: 0.724 - ETA: 1s - loss: 1.0559 - mse: 1.0559 - mae: 0.709 - ETA: 1s - loss: 1.0343 - mse: 1.0343 - mae: 0.700 - ETA: 1s - loss: 0.9869 - mse: 0.9869 - mae: 0.680 - ETA: 1s - loss: 0.9379 - mse: 0.9379 - mae: 0.659 - ETA: 0s - loss: 0.8947 - mse: 0.8947 - mae: 0.641 - ETA: 0s - loss: 0.8550 - mse: 0.8550 - mae: 0.623 - ET

In [21]:
y_val = model.predict(x_test)

In [22]:
mae = metrics.mean_absolute_error(y_test, y_val)
mse = metrics.mean_squared_error(y_test, y_val)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_val))
rsquared = metrics.r2_score(y_test, y_val)

In [37]:
model_stat_dict = {
    'Mean Absolute Error': [mae],
    'Mean Squared Error': [mse],
    'Root Mean Squared Error': [rmse],
    'R-squared score': [rsquared]
}

In [39]:
model_stat = pd.DataFrame(model_stat_dict)

In [41]:
model_stat.to_csv("./Model_Statistics.csv")

In [42]:
y_pred = model.predict(test_x)

In [43]:
final_df =  pd.DataFrame()

In [44]:
final_df["COORD_X"] = test_X_Coord
final_df["COORD_Y"] = test_Y_Coord
final_df["Y"] = y_pred

In [45]:
final_df.to_csv("./Output.csv")

In [46]:
s = train_df["Y"].describe()

In [47]:
stat = pd.DataFrame(s)

In [48]:
stat = stat.drop(["count"])

In [49]:
stat.to_csv("./Statistics.csv")